In [ ]:
import os
import json

# Paths (adjust if needed)
eval_dir = "/home/jianingy/research/fast3r/notebooks/RealEstate10K_eval"
test_file = "/home/jianingy/research/fast3r/notebooks/re10k_test_1800.txt"

# Read the list of scene IDs into a set for fast lookup
with open(test_file, "r") as f:
    test_scenes = set(line.strip() for line in f if line.strip())

# Metrics we want to average
metrics = ["RRA_at_5", "RRA_at_15", "RRA_at_30",
           "RTA_at_5", "RTA_at_15", "RTA_at_30",
           "mAA_30"]

# Initialize accumulators
acc = {m: 0.0 for m in metrics}
count = 0

# Go through each file in the eval_dir
for filename in os.listdir(eval_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(eval_dir, filename)

        # Read JSON text.  If it's single-quoted JSON, handle with `json.loads` carefully
        # or use `ast.literal_eval`. If your data uses proper double-quoted JSON, regular json.loads will do.
        with open(filepath, "r") as ff:
            # If your data is strictly valid JSON, you can do:
            # data = json.load(ff)
            # If it has single quotes or other slight deviations, you can do:
            text = ff.read()
            # Convert single quotes to double quotes if your data is actually single-quoted
            text = text.replace("'", "\"")
            data = json.loads(text)

        video_name = data.get("video_name")

        # Check if this video is in our test set
        if video_name in test_scenes:
            # Accumulate each metric
            for m in metrics:
                acc[m] += data.get(m, 0.0)
            count += 1

# Compute averages if we found matches
if count > 0:
    for m in metrics:
        acc[m] /= count

print(f"Number of matched scenes: {count}")
print("Averaged metrics across matched scenes:")
for m in metrics:
    print(f"{m}: {acc[m]:.6f}")


In [ ]:
# find the worst 10 scenes on mAA_30
worst_10 = []
for filename in os.listdir(eval_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(eval_dir, filename)

        # Read JSON text.  If it's single-quoted JSON, handle with `json.loads` carefully
        # or use `ast.literal_eval`. If your data uses proper double-quoted JSON, regular json.loads will do.
        with open(filepath, "r") as ff:
            # If your data is strictly valid JSON, you can do:
            # data = json.load(ff)
            # If it has single quotes or other slight deviations, you can do:
            text = ff.read()
            # Convert single quotes to double quotes if your data is actually single-quoted
            text = text.replace("'", "\"")
            data = json.loads(text)

        video_name = data.get("video_name")

        # Check if this video is in our test set
        if video_name in test_scenes:
            worst_10.append((video_name, data.get("mAA_30", 0.0)))

print("Worst 10 scenes on mAA_30:")
worst_10.sort(key=lambda x: x[1])
for video_name, score in worst_10[:10]:
    print(f"{video_name}: {score:.6f}")